In [2]:
from pymongo import MongoClient
from bson.objectid import ObjectId
from bson.son import SON

In [3]:
client = MongoClient("mongodb://10.1.1.9:27017")
db = client["Neo4Food"]
orders = db["Orders"]
restaurants = db["Restaurants"]
users = db["Users"]

In [19]:
doppioni = restaurants.aggregate([{"$group": {"_id": "$email", "count": {"$sum": 1}}},{"$match": {"count": {"$gt": 1}}},{"$sort": {"count": 1}}])
for d in doppioni:
    count = d['count']
    # print(f"Email: {d['_id']} | Count: {count}")
    print("-----")
    for i in range(count):
        email = d['_id'].replace('@',f'{i}@')
        print(f"Old: {d['_id']} | New: {email}")
        # restaurants.update_one({"email": d['_id']},{"$set": {"email": email}})
        break
    break

-----
Old: Subway211@america.us | New: Subway2110@america.us
